# **Diagnóstico de Diabetes com Redes Neurais**

Nesta atividade, vamos trabalhar com um problema aplicado de **classificação binária**: prever se uma pessoa possui ou não diabetes com base em um conjunto de variáveis clínicas.

[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)

---

## **Contexto**

- O dataset utilizado é o **Pima Indians Diabetes Dataset**, coletado originalmente pelo Instituto Nacional de Diabetes e Doenças Digestivas e Renais dos Estados Unidos.
- Ele contém registros de mulheres com pelo menos 21 anos de idade da população Pima, um grupo étnico nativo norte-americano com alta incidência de diabetes tipo 2.

## **Objetivo**

O objetivo é treinar uma MLP para prever a presença de diabetes a partir de atributos fisiológicos e laboratoriais.

## **Variáveis de entrada**

Cada observação contém os seguintes atributos:

1. **Pregnancies**, number of times pregnant: Variável discreta.
2. **Glucose**, plasma glucose concentration after 2 hours in an oral glucose tolerance test: Variável contínua.
3. **BloodPressure**, diastolic blood pressure, in mm Hg: Variável contínua.
4. **SkinThickness**, triceps skin fold thickness, in mm: Variável contínua.
5. **Insulin**, 2-hour serum insulin, in μU/mL: Variável contínua.
6. **BMI**, body mass index, weight in kg/(height in m)²: Variável contínua.
7. **DiabetesPedigreeFunction**, family history function: Variável contínua.
8. **Age**, in years: : Variável discreta.

## **Variáveis de saída (Target)**

- **Outcome = 1**: Diabetic
- **Outcome = 0**: Non-diabetic


# **Arquitetura da Rede Neural e Procedimentos Adotados**

---

## **Arquitetura da rede**

A estrutura da rede foi definida como:

- **Entrada**: $8$ variáveis de entrada (padronizadas).
- **1ª camada oculta**: $6$ neurônios com ativação $\phi(z)$.
- **2ª camada oculta**: $3$ neurônios com ativação $\phi(z)$.
- **Camada de saída**: $2$ neurônios com ativação **Softmax**, representando as probabilidades associadas a cada classe (saída codificada em one-hot)

## **Funções de Ativação**

- Nas **camadas ocultas**, utilizamos a função Rectified Linear Unit (**ReLU**):
  $$
  \phi(z) = \max(0, z),
  $$
  computacionalmente eficiente e ajuda a evitar o problema de saturação presente em funções como a sigmoide.

- Na **camada de saída**, utilizamos a função **Softmax**:
  $$
  \text{softmax}(z_j) = \frac{e^{z_j}}{\sum_{k} e^{z_k}}
  $$

## **Função de Custo**

Como a saída está codificada em **one-hot**, adotamos a **cross-entropy categórica** como função de custo:
$$
\mathcal{L}(y, \hat{y}) = - \frac{1}{n} \sum_{i=1}^n \sum_{j=1}^{2} y_{ij} \log(\hat{y}_{ij})
$$

## **Procedimento de Otimização**

O treinamento foi realizado utilizando o algoritmo de **descida do gradiente clássica (batch)**:

- Os gradientes foram computados por meio do algoritmo de **backpropagation**. Os pesos foram atualizados de forma simultânea com base no erro de todo o conjunto de treino, com taxa de aprendizado $\eta$.

## **Tratamento dos Dados**

Antes do treinamento, os dados foram processados da seguinte forma:

- **Remoção de entradas inválidas**, com valores zero biologicamente implausíveis.
- **Padronização** das variáveis de entrada via z-score.
- **Codificação one-hot** da target (binária).
- **Divisão em conjuntos** de treino (70%), validação (15%) e teste (15%).

## **Avaliação**

Durante o treinamento, foram monitoradas:

- A **função de perda** (cross-entropy) em treino e validação.
- A **acurácia** em ambos os conjuntos.

Após o treinamento, o modelo foi avaliado no **conjunto de teste** por meio de:
- Matriz de confusão apresentando métricas de acurácia, precisão, recall e $F_1$-score.


In [1]:
# @title MLP com ReLU nas camadas ocultas e Softmax na saída (2 camadas ocultas)

import matplotlib.pyplot as plt
import networkx as nx

# Criar grafo direcionado
G = nx.DiGraph()

# Camadas conforme a descrição
input_layer = [f'x_{i+1}' for i in range(8)]
hidden_layer1 = [f'h^1_{i+1}' for i in range(6)]
hidden_layer2 = [f'h^2_{i+1}' for i in range(3)]
output_layer = ['{y}_0', '{y}_1']

# Lista de camadas
layers = [input_layer, hidden_layer1, hidden_layer2, output_layer]
positions = {}
labels = {}
layer_dist = 2.0
node_dist = 1.0

# Posicionamento dos nós
for i, layer in enumerate(layers):
    for j, node in enumerate(layer):
        G.add_node(node)
        positions[node] = (i * layer_dist, -j * node_dist)
        labels[node] = f"${node}$"

# Conectar camadas
def connect_layers(layer1, layer2):
    for u in layer1:
        for v in layer2:
            G.add_edge(u, v)

connect_layers(input_layer, hidden_layer1)
connect_layers(hidden_layer1, hidden_layer2)
connect_layers(hidden_layer2, output_layer)

# Desenho
plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(G, positions, node_color='lightgray', node_size=1000)
nx.draw_networkx_edges(G, positions, arrows=True, arrowstyle='-|>', width=1)
nx.draw_networkx_labels(G, positions, labels, font_size=12)
plt.title('Arquitetura: Entrada (8) → Oculta1 (6, ReLU) → Oculta2 (3, ReLU) → Saída (2, Softmax)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# @title Importação dos dados
import pandas as pd

# URL do dataset
url = "https://raw.githubusercontent.com/pcbrom/perceptron-mlp-cnn/refs/heads/main/data/diabetes.csv"

# Carregar o dataset
df = pd.read_csv(url)

# Verificar dimensões
print(f"Shape do dataset: {df.shape}")

# Visualizar as primeiras linhas
display(df.head())

# Verificar estatísticas básicas
display(df.describe())

In [ ]:
# @title Verificação de valores zero inválidos (biologicamente implausíveis)

# Colunas que não devem conter zero
cols_with_invalid_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Contagem de zeros por coluna
invalid_zeros = (df[cols_with_invalid_zeros] == 0).sum()

print("Contagem de valores igual a zero (potencialmente inválidos):")
display(invalid_zeros)


In [ ]:
# @title Remoção de linhas com valores zero inválidos

# Colunas que não devem conter zero
cols_with_invalid_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Filtrar apenas linhas com valores válidos (não-zero) nas colunas indicadas
df_clean = df.copy()
for col in cols_with_invalid_zeros:
    df_clean = df_clean[df_clean[col] != 0]

# Verificar nova dimensão do conjunto de dados
print(f"Shape após remoção: {df_clean.shape}")


In [ ]:
# @title Padronização antes da divisão e One-Hot Encoding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Separar variáveis explicativas e alvo
X = df_clean.drop(columns='Outcome').values
y = df_clean['Outcome'].values.reshape(-1, 1)  # necessário para o encoder

# Padronizar (z-score)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# 70% treino, 30% temporário (validação + teste)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y_encoded, test_size=0.30, stratify=y, random_state=42
)

# 15% validação, 15% teste
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

# Verificações
print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de validação: {X_val.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}\n")


In [ ]:
# Hiperparâmetros
eta = 0.0001     # teste com outros valores (exercício)
epochs = 1500    # teste com outros valores (exercício)

In [ ]:
# @title MLP com ReLU nas camadas ocultas e Softmax na saída (2 camadas ocultas) DG

import numpy as np
import matplotlib.pyplot as plt

# Arquitetura
n_input = X_train.shape[1]
n_hidden1 = 6
n_hidden2 = 3
n_output = 2  # saída one-hot

# Funções de ativação
def relu(z):
    return np.maximum(0, z)

def drelu(z):
    return (z > 0).astype(float)

def softmax(z):
    z_stable = z - np.max(z, axis=1, keepdims=True)  # estabilidade numérica
    exp_z = np.exp(z_stable)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def cross_entropy(y_hat, y_true):
    eps = 1e-10
    return -np.mean(np.sum(y_true * np.log(y_hat + eps), axis=1))

# Inicialização
np.random.seed(42)
W1 = np.random.randn(n_input, n_hidden1)
b1 = np.zeros((1, n_hidden1))
W2 = np.random.randn(n_hidden1, n_hidden2)
b2 = np.zeros((1, n_hidden2))
W3 = np.random.randn(n_hidden2, n_output)
b3 = np.zeros((1, n_output))

# Histórico
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []

for epoch in range(epochs):
    # --- Forward (treino)
    Z1 = X_train @ W1 + b1
    A1 = relu(Z1)
    Z2 = A1 @ W2 + b2
    A2 = relu(Z2)
    Z3 = A2 @ W3 + b3
    A3 = softmax(Z3)

    # --- Forward (validação)
    Z1v = X_val @ W1 + b1
    A1v = relu(Z1v)
    Z2v = A1v @ W2 + b2
    A2v = relu(Z2v)
    Z3v = A2v @ W3 + b3
    A3v = softmax(Z3v)

    # --- Loss
    train_loss = cross_entropy(A3, y_train)
    val_loss = cross_entropy(A3v, y_val)
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)

    # --- Acurácia
    train_pred = np.argmax(A3, axis=1)
    val_pred = np.argmax(A3v, axis=1)
    train_acc = np.mean(train_pred == np.argmax(y_train, axis=1))
    val_acc = np.mean(val_pred == np.argmax(y_val, axis=1))
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)

    # --- Backpropagation
    dZ3 = (A3 - y_train)  # derivada da softmax + cross-entropy
    dW3 = A2.T @ dZ3
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dZ2 = (dZ3 @ W3.T) * drelu(Z2)
    dW2 = A1.T @ dZ2
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dZ1 = (dZ2 @ W2.T) * drelu(Z1)
    dW1 = X_train.T @ dZ1 # usa todos os dados simultaneamente
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    # --- Atualização
    W3 -= eta * dW3
    b3 -= eta * db3
    W2 -= eta * dW2
    b2 -= eta * db2
    W1 -= eta * dW1
    b1 -= eta * db1

    # Log ocasional
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# --- Gráficos
plt.figure(figsize=(8, 4))

# Loss
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Val Loss')
plt.title('Evolução da Função de Perda (Cross-Entropy)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

# Acurácia
plt.subplot(1, 2, 2)
plt.plot(train_acc_history, label='Train Acc')
plt.plot(val_acc_history, label='Val Acc')
plt.title('Evolução da Acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.tight_layout()
plt.show()


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# Forward no conjunto de teste
Z1t = X_test @ W1 + b1
A1t = relu(Z1t)
Z2t = A1t @ W2 + b2
A2t = relu(Z2t)
Z3t = A2t @ W3 + b3
A3t = softmax(Z3t)

# Predição: classe com maior probabilidade
y_pred_test = np.argmax(A3t, axis=1)
y_true_test = np.argmax(y_test, axis=1)

# Acurácia
acc_test = accuracy_score(y_true_test, y_pred_test)
print(f"Acurácia no conjunto de teste: {acc_test:.4f}")

# Matriz de confusão
cm = confusion_matrix(y_true_test, y_pred_test)
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusão (Teste)')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

# Relatório completo (opcional)
print("\nRelatório de Classificação:")
print(classification_report(y_true_test, y_pred_test, digits=4, zero_division=1))

if epoch == epochs - 1:
    print("Peso final W1[0,0]:", W1[0, 0])


In [ ]:
# Hiperparâmetros
eta = 0.0001     # teste com outros valores (exercício)
epochs = 1500    # teste com outros valores (exercício)

In [ ]:
# @title MLP com ReLU nas camadas ocultas e Softmax na saída (SGD - 2 camadas ocultas)

import numpy as np
import matplotlib.pyplot as plt

# Arquitetura
n_input = X_train.shape[1]
n_hidden1 = 6
n_hidden2 = 3
n_output = 2

# Funções
def relu(z): return np.maximum(0, z)
def drelu(z): return (z > 0).astype(float)
def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
def cross_entropy(y_hat, y_true):
    eps = 1e-10
    return -np.mean(np.sum(y_true * np.log(y_hat + eps), axis=1))

# Inicialização
np.random.seed(42)
W1 = np.random.randn(n_input, n_hidden1)
b1 = np.zeros((1, n_hidden1))
W2 = np.random.randn(n_hidden1, n_hidden2)
b2 = np.zeros((1, n_hidden2))
W3 = np.random.randn(n_hidden2, n_output)
b3 = np.zeros((1, n_output))

# Histórico
train_loss_history, val_loss_history = [], []
train_acc_history, val_acc_history = [], []

for epoch in range(epochs):
    # --- Treinamento com SGD
    for i in range(X_train.shape[0]):
        xi = X_train[i:i+1]
        yi = y_train[i:i+1]

        # Forward
        Z1 = xi @ W1 + b1
        A1 = relu(Z1)
        Z2 = A1 @ W2 + b2
        A2 = relu(Z2)
        Z3 = A2 @ W3 + b3
        A3 = softmax(Z3)

        # Backprop
        dZ3 = (A3 - yi)
        dW3 = A2.T @ dZ3
        db3 = dZ3

        dZ2 = (dZ3 @ W3.T) * drelu(Z2)
        dW2 = A1.T @ dZ2
        db2 = dZ2

        dZ1 = (dZ2 @ W2.T) * drelu(Z1)
        dW1 = xi.T @ dZ1 # usa apenas uma amostra por vez
        db1 = dZ1

        # Atualização
        W3 -= eta * dW3
        b3 -= eta * db3
        W2 -= eta * dW2
        b2 -= eta * db2
        W1 -= eta * dW1
        b1 -= eta * db1

    # --- Avaliação por época (para gráfico)
    # Treino
    Z1 = X_train @ W1 + b1
    A1 = relu(Z1)
    Z2 = A1 @ W2 + b2
    A2 = relu(Z2)
    Z3 = A2 @ W3 + b3
    A3 = softmax(Z3)
    train_loss = cross_entropy(A3, y_train)
    train_acc = np.mean(np.argmax(A3, axis=1) == np.argmax(y_train, axis=1))

    # Validação
    Z1v = X_val @ W1 + b1
    A1v = relu(Z1v)
    Z2v = A1v @ W2 + b2
    A2v = relu(Z2v)
    Z3v = A2v @ W3 + b3
    A3v = softmax(Z3v)
    val_loss = cross_entropy(A3v, y_val)
    val_acc = np.mean(np.argmax(A3v, axis=1) == np.argmax(y_val, axis=1))

    # Registro
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)

    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# --- Gráficos
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Val Loss')
plt.title('Perda (SGD)')
plt.xlabel('Época')
plt.ylabel('Cross-Entropy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_acc_history, label='Train Acc')
plt.plot(val_acc_history, label='Val Acc')
plt.title('Acurácia (SGD)')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.tight_layout()
plt.show()


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# --- Forward completo no conjunto de teste
Z1t = X_test @ W1 + b1
A1t = relu(Z1t)
Z2t = A1t @ W2 + b2
A2t = relu(Z2t)
Z3t = A2t @ W3 + b3
A3t = softmax(Z3t)

# --- Predição final
y_pred_test = np.argmax(A3t, axis=1)
y_true_test = np.argmax(y_test, axis=1)

# --- Acurácia
acc_test = accuracy_score(y_true_test, y_pred_test)
print(f"Acurácia no conjunto de teste: {acc_test:.4f}")

# --- Matriz de confusão
cm = confusion_matrix(y_true_test, y_pred_test)
plt.figure(figsize=(4, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusão (Teste)')
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.tight_layout()
plt.show()

# --- Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_true_test, y_pred_test, digits=4, zero_division=1))

if epoch == epochs - 1:
    print("Peso final W1[0,0]:", W1[0, 0])

# **Observação**

- O seed é o mesmo $\implies$ Os pesos iniciais são iguais.
- A taxa de aprendizado é muito baixa.
- O dataset é pequeno, e o número de épocas alto suaviza o ruído estocástico.

Ainda que os resultados sejam "iguais" a uma promeira vista, os processos são diferentes e podemos perceber olhando os pesos após o treinamento, por exeplo,

- GD: Peso final $W1[0,0]: 0.8271388985463952$.
- SGD: Peso final $W1[0,0]: 0.8264546783200991$.

# **Noções de Redes Convolucionais (CNN) com PyTorch**

- As redes convolucionais, Convolutional Neural Networks, (CNNs) são especialmente eficazes para tarefas que envolvem **dados com estrutura espacial**, como imagens.
- Sua principal vantagem é a capacidade de **aprender padrões locais** por meio de **filtros convolucionais**, reduzindo o número de parâmetros em comparação com redes totalmente conectadas.

---

## **Arquitetura de uma CNN**

[Site recomendado: Arquitetura de uma CNN](https://alexlenail.me/NN-SVG/LeNet.html)

A imagem abaixo ilustra todas as etapas de uma CNN típica, desde a entrada da imagem até a saída de classificação:

![CNN](https://cdn.hashnode.com/res/hashnode/image/upload/v1722198375823/1c6c0f55-6748-4c25-ad0d-4cff68b2a5f4.png?auto=compress,format&format=webp)
Fonte da figura: sisirdhakal.hashnode.dev

**Etapas Representadas:**
1. **Entrada**: Imagem de entrada (por exemplo, 28×28 pixels em tons de cinza).
2. **Camadas Convolucionais**: Aplicam filtros para extrair características locais.
3. **Funções de Ativação (ReLU)**: Introduzem não linearidade.
4. **Camadas de Pooling (Max Pooling)**: Reduzem a dimensionalidade espacial.
5. **Camada de Flattening**: Transforma os mapas de ativação em um vetor unidimensional.
6. **Camadas Totalmente Conectadas**: Realizam a classificação com base nas características extraídas.
7. **Camada de Saída (Softmax)**: Fornece as probabilidades associadas a cada classe.

## **Componentes fundamentais de uma CNN**

1. **Camada Convolucional (`nn.Conv2d`)** Aplica um conjunto de filtros (ou *kernels*) sobre a imagem de entrada. Cada filtro desliza sobre a imagem e gera um mapa de ativação.
  - **Quando usar:** Sempre que você quiser extrair padrões locais (bordas, texturas, formas) de imagens ou dados com estrutura espacial.
  -   **Função:** Aprende filtros treináveis que identificam características locais (ex: bordas verticais, linhas diagonais, texturas).

  ```python
  nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
  ```
  - `in_channels=1` Número de canais de entrada. Uma imagem em escala de cinza tem 1 canal, uma imagem RGB tem 3.
  - `out_channels=8` Número de filtros (kernels) que a camada irá aprender. Cada filtro gera um mapa de ativação. Com 8 filtros, a saída terá 8 canais.
  - `kernel_size=3` Tamanho dos filtros convolucionais: $3 \times 3$.Isso significa que cada filtro examina uma vizinhança $3 \times 3$ da imagem em cada passo.

2. **Função de ativação, exemplo `(nn.ReLU)`**
  - **Quando usar:** Sempre após camadas convolucionais ou lineares, para adicionar não linearidade ao modelo.
  - **Função:** Permite que a rede aprenda funções não lineares, aumentando seu poder de representação.

3. **Pooling, exemplo `(nn.MaxPool2d)`** Reduz a dimensionalidade espacial (resolução), mantendo os valores máximos locais.
  - **Quando usar:** Logo após uma convolução + ativação, para reduzir a resolução espacial (dimensão da imagem) e manter as informações mais fortes.
  - **Função:** Reduz a complexidade e o número de parâmetros, tornando a rede mais eficiente e mais robusta a pequenas variações de posição (invariância local).

  ```python
  nn.MaxPool2d(kernel_size=2, stride=2)
  ```
  - `kernel_size=2` Define o tamanho da janela de pooling: $2 \times 2$. A camada seleciona o maior valor dentro de cada janela $2 \times 2$ da entrada.
  - `stride=2` Define o passo de deslocamento da janela: pula de 2 em 2 pixels. Isso reduz a dimensão da imagem pela metade (subamostragem).



### **Dataset MNIST**

- O Modified National Institute of Standards and Technology (MNIST) é um dos conjuntos de dados mais clássicos e utilizados para a introdução ao aprendizado profundo e CNN.
- Ele contém imagens de **dígitos manuscritos** de 0 a 9, escritas por diferentes pessoas.

[Site recomendado: Arquitetura de uma CNN](https://alexlenail.me/NN-SVG/LeNet.html)

A imagem abaixo ilustra todas as etapas de uma CNN típica, desde a entrada da imagem até a saída de classificação:

![MNIST](https://www.mdpi.com/applsci/applsci-09-03169/article_deploy/html/images/applsci-09-03169-g001.png)
Fonte da figura: https://www.mdpi.com/2076-3417/9/15/3169

---

#### **Características do MNIST**

- **Número de classes**: 10 (dígitos de `0` a `9`)
- **Número de imagens de treino**: 60.000
- **Número de imagens de teste**: 10.000
- **Formato das imagens**:  
  - Tamanho: $28 \times 28$ pixels  
  - Canais: 1 (tons de cinza, escala de 0 a 255)
- **Tipo dos dados**: imagens rotuladas

#### **Objetivo da Tarefa**

Treinar um modelo capaz de **reconhecer automaticamente o dígito manuscrito** em uma imagem, mesmo com diferentes estilos de escrita.


In [ ]:
# @title CNN - Exemplo com MNIST

# Importação das bibliotecas principais
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Configuração do dispositivo: GPU (se disponível) ou CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pré-processamento das imagens
# - Converte para tensor
# - Normaliza com média e desvio padrão da base MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Carregamento do conjunto de treino completo e de teste
full_train = datasets.MNIST(root='.', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='.', train=False, transform=transform)

# Divisão do conjunto de treino em treino (50.000) e validação (10.000)
train_dataset, val_dataset = random_split(full_train, [50000, 10000])

# Criação dos DataLoaders (permitem leitura em mini-batches)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=1000, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Definição da arquitetura da rede convolucional (CNN)
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Primeira camada convolucional: 1 canal de entrada, 8 filtros 3x3
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3)
        # Segunda camada convolucional: 8 canais de entrada, 16 filtros 3x3
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        # Camada de pooling para reduzir a dimensionalidade (janela 2x2)
        self.pool = nn.MaxPool2d(2, 2)
        # Camada totalmente conectada (após achatar para vetor)
        self.fc1 = nn.Linear(16 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 10)  # 10 classes (dígitos de 0 a 9)

    def forward(self, x):
        # Aplicação da sequência: convolução → ReLU → pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # Achata a saída para vetor unidimensional
        x = x.view(-1, 16 * 5 * 5)
        # Camadas densas com ativação ReLU
        x = F.relu(self.fc1(x))
        # Saída final sem ativação (CrossEntropy já aplica Softmax internamente)
        return self.fc2(x)

# Instancia o modelo, define a função de perda e o otimizador
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()  # Perda para classificação multiclasse
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Parâmetros de treinamento
epochs = 5
train_loss_hist, val_loss_hist = [], []
train_acc_hist, val_acc_hist = [], []

# Loop de treinamento
for epoch in range(epochs):
    model.train()  # Modo treino
    train_loss, train_correct = 0, 0

    # Treinamento por mini-batches
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()           # Zera gradientes anteriores
        out = model(x)                  # Faz a previsão
        loss = criterion(out, y)        # Calcula o erro
        loss.backward()                 # Backpropagation
        optimizer.step()                # Atualiza os pesos

        train_loss += loss.item() * x.size(0)
        train_correct += (out.argmax(1) == y).sum().item()

    # Validação (sem atualização de pesos)
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_correct += (out.argmax(1) == y).sum().item()

    # Armazena resultados
    train_loss_hist.append(train_loss / len(train_dataset))
    val_loss_hist.append(val_loss / len(val_dataset))
    train_acc_hist.append(train_correct / len(train_dataset))
    val_acc_hist.append(val_correct / len(val_dataset))

    # Exibe progresso
    print(f"Época {epoch+1}/{epochs} | Loss Treino: {train_loss_hist[-1]:.4f} | Loss Val: {val_loss_hist[-1]:.4f} | Acc Treino: {train_acc_hist[-1]*100:.2f}% | Acc Val: {val_acc_hist[-1]*100:.2f}%")

# Gráficos de desempenho por época
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
plt.plot(train_loss_hist, label="Treino")
plt.plot(val_loss_hist, label="Validação")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Função de Perda")
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_acc_hist, label="Treino")
plt.plot(val_acc_hist, label="Validação")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.title("Acurácia")
plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
# @title Avaliação no conjunto de teste

# Avaliação no conjunto de teste
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        out = model(x)
        preds = out.argmax(1).cpu()
        y_true.extend(y.tolist())
        y_pred.extend(preds.tolist())

# Relatório de classificação com precisão, recall e F1-score
print("Relatório de Classificação:")
print(classification_report(y_pred, y_true, digits=4))

# Matriz de confusão
conf = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(conf, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de Confusão - Conjunto de Teste")
plt.xlabel("Classe Real")
plt.ylabel("Classe Predita")
plt.show()